In [4]:
import numpy as np
import sys, os
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
from skimage.color import rgb2gray
from skimage import measure
import cv2
from skimage import morphology
from skimage.morphology import disk, square, dilation, erosion
from skimage import data
from skimage.feature import Cascade
from EyeCenterLocator import EyeCenterLocator
from FaceAligner import FaceAligner
import imutils
from scipy import signal
from skimage import filters
from skimage import feature
import skimage.io as io
from scipy import fftpack
from scipy.signal import convolve2d
from skimage.util import random_noise
from skimage.exposure import rescale_intensity
from PIL import Image
import requests

In [5]:
kEyePercentTop = 25
kEyePercentSide = 8
kEyePercentHeight = 30
kEyePercentWidth = 35

In [6]:
def cropImage(image, p_width, p_hieght):
    width = image.shape[0]
    height = image.shape[1]
    left = int((width - width*p_width)//2)
    top = int((height - height*p_hieght)//2)
    right = int((width +  width*p_width)//2)
    bottom = int((height + height*p_hieght)//2)
    
    return image[left:right, top:bottom]

# Testing using webcam

In [8]:
haar_file = 'haarcascade_frontalface_default.xml'
webcam = cv2.VideoCapture(0)
face_cascade = cv2.CascadeClassifier(haar_file)

leftEyeCenter = (0, 0)
rightEyeCenter = (0, 0)
leftEyeRegionStart = (0, 0)
rightEyeRegionStart= (0, 0)
face_rect = (0, 0, 0, 0)
fa = FaceAligner(desiredFaceWidth=255)
while True:
    (_, img_BGR) = webcam.read()
    
    img_GRAY = cv2.cvtColor(img_BGR, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(img_GRAY)
    
    if len(faces) == 0:
        continue
        
    for (x, y, w, h) in faces:
        face = img_BGR[y :y + h , x :x + w ]
        face = cropImage(face, 0.9, 0.8)
        leftEyeCenter, rightEyeCenter = getEyesCenter(face)
        face = cv2.circle(face, (leftEyeCenter[0], leftEyeCenter[1]), 1, (255,0,255), -1)
        face = cv2.circle(face, (rightEyeCenter[0], rightEyeCenter[1]), 1, (255,0,255), -1)
        
        imageAligned = fa.align(img_BGR, rightEyeCenter, leftEyeCenter)
        detected_faces = face_cascade.detectMultiScale(cv2.cvtColor(imageAligned, cv2.COLOR_BGR2GRAY))
        if len(detected_faces) > 0:
            
            (xA, yA, wA, hA) = detected_faces[0]
            faceAligned = imageAligned[yA :yA + hA , xA :xA + wA]
            
            cv2.imshow('Face before 2D Alignment', face)
            
            cv2.imshow('Image after 2D Alignment', faceAligned)
            cv2.imshow('Face after 2D Alignment', imageAligned)
            #cv2.imshow('Image before 2D Alignment', img_BGR)
        break
    
    key = cv2.waitKey(5)    
    if key == 27:
        break
    
cv2.destroyAllWindows()
webcam.release()

In [12]:
cv2.destroyAllWindows()
webcam.release()

# Testin Using Stream

In [30]:
# Replace the below URL with your own. Make sure to add "/shot.jpg" at last.
url = "http://192.168.1.3:8080/shot.jpg"
haar_file = 'haarcascade_frontalface_default.xml'
face_cascade = cv2.CascadeClassifier(haar_file)

fa = FaceAligner(desiredFaceWidth=255)

leftEyeCenter = (0, 0)
rightEyeCenter = (0, 0)
leftEyeRegionStart = (0, 0)
rightEyeRegionStart= (0, 0)
face_rect = (0, 0, 0, 0)

# While loop to continuously fetching data from the Url
while True:
    img_resp = requests.get(url)
    img_arr = np.array(bytearray(img_resp.content), dtype=np.uint8)
    img_BGR = cv2.imdecode(img_arr, -1)
    img_BGR = imutils.resize(img_BGR, width=480, height=960)
    #cv2.imshow("Android_cam", img)
    
    
    img_GRAY = cv2.cvtColor(img_BGR, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(img_GRAY)
    
    if len(faces) == 0:
        continue
        
    for (x, y, w, h) in faces:
        face = img_BGR[y :y + h , x :x + w ]
        face = cropImage(face, 0.9, 0.8)
        leftEyeCenter, rightEyeCenter = getEyesCenter(face)
        face = cv2.circle(face, (leftEyeCenter[0], leftEyeCenter[1]), 1, (255,0,255), -1)
        face = cv2.circle(face, (rightEyeCenter[0], rightEyeCenter[1]), 1, (255,0,255), -1)
        
        imageAligned = fa.align(img_BGR, rightEyeCenter, leftEyeCenter)
        detected_faces = face_cascade.detectMultiScale(cv2.cvtColor(imageAligned, cv2.COLOR_BGR2GRAY))
        if len(detected_faces) > 0:
            
            (xA, yA, wA, hA) = detected_faces[0]
            faceAligned = imageAligned[yA :yA + hA , xA :xA + wA]
            
            cv2.imshow('Face before 2D Alignment', face)
            
            cv2.imshow('Image after 2D Alignment', faceAligned)
            cv2.imshow('Face after 2D Alignment', imageAligned)
            #cv2.imshow('Image before 2D Alignment', img_BGR)
        break
        
    # Press Esc key to exit
    if cv2.waitKey(1) == 27:
        break
  
cv2.destroyAllWindows()

ConnectionError: HTTPConnectionPool(host='192.168.1.3', port=8080): Max retries exceeded with url: /shot.jpg (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x00000231401266D0>: Failed to establish a new connection: [WinError 10060] A connection attempt failed because the connected party did not properly respond after a period of time, or established connection failed because connected host has failed to respond'))